In [1]:
import torch
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import os


In [2]:
def show_files(folder):
    for ind in range(len(folder)):
        print(f"{ind:3}: {folder[ind]}")


In [3]:
img_folder = "./images/"
arr = os.listdir(img_folder)
arr = [f for f in arr if os.path.isfile(f"{img_folder}{f}")]
arr.sort()
show_files(arr)


  0: baby.jpg
  1: cafezin.jpg
  2: chuva.jpg
  3: end.jpg
  4: end_cervejada.jpg
  5: fatos.png
  6: festa_espelho.jpg
  7: fred.jpg
  8: gato_move.jpg
  9: gato_rack.jpg
 10: gato_tv.jpg
 11: jojo.jpg
 12: o_palhaco.jpg
 13: rick_chuva.jpg
 14: rick_end_cervejada.jpg
 15: rick_end_fantasia.jpg
 16: rieck.jpg
 17: serec.jpg
 18: serio.jpg
 19: simba_intenso.jpg
 20: sky.jpg
 21: veloso.jpg
 22: vlad.jpg


In [4]:
img_paths = []
for ind in range(len(arr)):
    path = f"{img_folder}{arr[ind]}"
    img_paths.append(path)

show_files(img_paths)


  0: ./images/baby.jpg
  1: ./images/cafezin.jpg
  2: ./images/chuva.jpg
  3: ./images/end.jpg
  4: ./images/end_cervejada.jpg
  5: ./images/fatos.png
  6: ./images/festa_espelho.jpg
  7: ./images/fred.jpg
  8: ./images/gato_move.jpg
  9: ./images/gato_rack.jpg
 10: ./images/gato_tv.jpg
 11: ./images/jojo.jpg
 12: ./images/o_palhaco.jpg
 13: ./images/rick_chuva.jpg
 14: ./images/rick_end_cervejada.jpg
 15: ./images/rick_end_fantasia.jpg
 16: ./images/rieck.jpg
 17: ./images/serec.jpg
 18: ./images/serio.jpg
 19: ./images/simba_intenso.jpg
 20: ./images/sky.jpg
 21: ./images/veloso.jpg
 22: ./images/vlad.jpg


In [5]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True)
# model.cuda()

results = model(img_paths, size=1280) # múltiplo de 32


Using cache found in /home/rick/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-5-19 Python-3.11.3 torch-2.0.1+cpu CPU



requirements: /home/rick/Documents/University/TCC/TCC/lib/python3.11/site-packages/requirements.txt not found, check failed.


Fusing layers... 
YOLOv5l6 summary: 476 layers, 76726332 parameters, 0 gradients
Adding AutoShape... 


In [6]:

tags = results.pandas().xyxy[7][["name"]].groupby("name").value_counts()
tag_list = list(zip(tags.index, tags))
tag_list[0]

res = pd.concat(results.pandas().xyxy)
tags_res = res[['name']].groupby('name').value_counts().index
tags_res = list(tags_res)
tags_res.append('group')
tags_res.append('trio')
tags_res.append('pair')
tags_res.sort()

tags_res


['backpack',
 'bed',
 'bench',
 'book',
 'bottle',
 'bus',
 'cake',
 'cat',
 'cell phone',
 'chair',
 'cup',
 'dining table',
 'dog',
 'group',
 'handbag',
 'pair',
 'person',
 'remote',
 'tie',
 'toilet',
 'trio',
 'tv']

In [7]:
DG = nx.DiGraph()


In [8]:
for i in range(len(arr)):
    tag = results.pandas().xyxy[i][["name"]].groupby("name").value_counts()
    tags = list(zip(tag.index, tag))

    img_name = arr[i]

    for j in range(len(tags)):
        tag_name = tags[j][0]
        tag_quant = tags[j][1]

        DG.add_edge(tag_name, img_name) # ligando nome da classe identificada com o nome do arquivo
        if tag_name == 'person':
            if tag_quant == 2:
                DG.add_edge('pair', img_name)
            elif tag_quant == 3:
                DG.add_edge('trio', img_name)
            if tag_quant > 3:
                DG.add_edge('group', img_name)
        if tag_name == 'cat' or tag_name == 'dog':
            DG.add_edge('pets', img_name)

    if len(tags) == 0:
        DG.add_node(img_name)


In [9]:
nx.draw_shell(DG, with_labels=True, font_weight='bold')

print(DG.number_of_nodes())
print(DG.number_of_edges())

plt.show()
plt.savefig("results/path.png")
plt.clf()


45
66


In [10]:
pos = nx.bipartite_layout(DG, tags_res, aspect_ratio=9/16)
nx.draw(DG, pos, with_labels=True, font_weight='bold')
plt.savefig('results/columns_old.png')
plt.clf()


In [14]:
tag_options = {
    "node_color" : "red",
    "node_size" : 75,
    "font_size" : 12,
    "font_color" : "black",
    "horizontalalignment" : "right",
    "verticalalignment" : "bottom",
}

photo_options = {
    "node_color" : "blue",
    "node_size" : 50,
    "font_size" : 12,
    "font_color" : "black",
    "horizontalalignment" : "left",
    "verticalalignment" : "center",
}

edge_options = {
    "edge_color" : "black",
    "width" : 1.5,
    "alpha" : .75
}

plt.figure(figsize=(8, 6))
plt.axis([-0.65, 0.65, -1, 1])
plt.box(False)
pos = nx.bipartite_layout(DG, tags_res)
for node_pos in pos.values():
  node_pos[0] *= 0.4
  print(node_pos)


nx.draw_networkx_edges(DG, pos, **edge_options)

# TAGS
tags_dict = {n:n for n in DG if n in tags_res}
nx.draw_networkx(DG, pos, nodelist=tags_res, with_labels=True, labels=tags_dict, **tag_options)

# PHOTOS
photos_dict = {n:n for n in DG if n not in tags_res}
nx.draw_networkx(DG, pos, nodelist=list(set(DG.nodes) - set(tags_res)), with_labels=True, labels=photos_dict, **photo_options)


plt.show()
plt.savefig(f"results/columns.png")
plt.clf()


[       -0.4    -0.71875]
[       -0.4     -0.6503]
[       -0.4    -0.58185]
[       -0.4    -0.51339]
[       -0.4    -0.44494]
[       -0.4    -0.37649]
[       -0.4    -0.30804]
[       -0.4    -0.23958]
[       -0.4    -0.17113]
[       -0.4    -0.10268]
[       -0.4   -0.034226]
[       -0.4    0.034226]
[       -0.4     0.10268]
[       -0.4     0.17113]
[       -0.4     0.23958]
[       -0.4     0.30804]
[       -0.4     0.37649]
[       -0.4     0.44494]
[       -0.4     0.51339]
[       -0.4     0.58185]
[       -0.4      0.6503]
[       -0.4     0.71875]
[    0.36667    -0.71875]
[    0.36667    -0.65625]
[    0.36667    -0.59375]
[    0.36667    -0.53125]
[    0.36667    -0.46875]
[    0.36667    -0.40625]
[    0.36667    -0.34375]
[    0.36667    -0.28125]
[    0.36667    -0.21875]
[    0.36667    -0.15625]
[    0.36667    -0.09375]
[    0.36667    -0.03125]
[    0.36667     0.03125]
[    0.36667     0.09375]
[    0.36667     0.15625]
[    0.36667     0.21875]
[    0.36667